In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import os

In [87]:
data_path = os.getenv('DATA_PATH')
products = pd.read_csv(f'{data_path}/BaSalam.products.csv', low_memory=False)

In [ ]:
def outliers_number(data: pd.Series, iqr=True,  modified=False, threshold=3, threshold_modified=4.5):
    if iqr:
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        IQR = q3-q1

        return data[(data > q3+IQR*1.5) | (data < q1-IQR*1.5)].index
    else:
        if modified:
            median = data.median()
            mad = np.median(np.abs(data - median))
            
            modified_z_scores = 0.6745 * (data - median) / mad

            return data[np.abs(modified_z_scores) > threshold_modified].index
        else:
            z_score = (data - data.mean()) / data.std()
            return data[np.abs(data) > threshold].index

cols = products.columns.tolist()
miss_values = pd.Series([int(products[col].isna().sum()) for col in cols])
    
def outlier_len(iqr, modified):
    outlier_iqr = []
    for col in cols:
        if (type(products[col][0]) in [np.int64, np.float64]):
            outlier_iqr.append(len(outliers_number(products[col], iqr, modified)))
        else:
            outlier_iqr.append(np.nan)

    return pd.Series(outlier_iqr)

pd.DataFrame(index=products.columns.tolist(),
             data={
            'miss value': miss_values.values,
            'uniq': products.nunique(),
            "type": products.dtypes.values,
            'outlier_iqr': outlier_len(True, False).values,
            'outlier_Z-score': outlier_len(False, False).values,
            'outlier_Z-score modified': outlier_len(False, True).values,
            'Most repeated': [products[col].value_counts().index[0] for col in cols]
            })

In [ ]:
sample = products.sample(n=100000)
sample.columns

In [ ]:
sample.groupby('categoryTitle')['_id'].count().sort_values(ascending=False)

In [ ]:
# filter products with rating count more than 1/5 of the max rating count
sample[sample['rating_count']>max(products['rating_count'])/5][['_id','rating_average', 'name', 'sales_count_week']].sort_values(by='rating_average')

In [ ]:
# products with count of their reviews
reviews_per_product = pd.read_csv('reviews_count_per_product.csv')
reviews_per_product.head()

In [ ]:
# number of products without any reviews
len(sample[~sample['_id'].isin(reviews_per_product['productId'])])/sample.shape[0]

In [ ]:
px.histogram(reviews_per_product['count'])

In [ ]:
# check the average rating of products with at least 10 reviews
p_atleast10comment = sample[sample['_id'].isin(reviews_per_product[reviews_per_product['count']>10]['productId'])]
p_atleast10comment[p_atleast10comment['rating_count'] > 100][['name', 'sales_count_week', 'rating_average', 'categoryTitle']].groupby(
    'categoryTitle').filter(lambda x: len(x)>5).groupby('categoryTitle')['rating_average'].agg(['mean', 'count']).sort_values(by='mean')

In [ ]:
# check the average rating of products with at least 5 reviews
print(sample[sample['categoryTitle'] == 'میوه خشک']['rating_average'].mean())
print(sample[(sample['categoryTitle'] == 'میوه خشک') & (sample['_id'].isin(reviews_per_product[reviews_per_product['count']>5]['productId']))]['rating_average'].mean())

In [ ]:
# check the average rating of categories with at least 50 products and at least 5 reviews

filtered = sample[(sample['categoryTitle'] != 'سایر') & (sample['rating_average']!=0) & (sample['_id'].isin(reviews_per_product[reviews_per_product['count']>5]['productId']))
                    ].groupby('categoryTitle').filter(lambda x: len(x)>5
                    ).groupby('categoryTitle')['rating_average'].mean().reset_index(name='rate'
                    ).sort_values(by='rate', ascending=True).head(30)

rating_by_cat_plot = pd.DataFrame({
    'category': filtered.categoryTitle.values,
    'rate': filtered.rate.values
})

fig = px.bar(rating_by_cat_plot, x='category', y='rate')
fig.update_layout(
    xaxis=dict(
        tickangle=-90,
        tickfont=dict(size=14, color='black') 
    )
)

fig.show()

In [ ]:
# check the number of rating count 
print(sum(sample[(sample['categoryTitle'] != 'سایر') & (sample['rating_average']!=0)]['rating_count']<10)/len(sample[(sample['categoryTitle'] != 'سایر') & (sample['rating_average']!=0)]['rating_count']))
fig = px.histogram(sample[(sample['categoryTitle'] != 'سایر') & (sample['rating_average']!=0)]['rating_count'])

fig.show()

In [ ]:
filtered = sample[(sample['categoryTitle'] != 'سایر') & (sample['rating_average']!=0
                )].groupby('categoryTitle').size().reset_index(name='product_count').sort_values(by='product_count', ascending=False).head(10)

rating_by_cat_plot = pd.DataFrame({
    'category': filtered.categoryTitle.values,
    'product_count': filtered.product_count.values
})

fig = px.bar(rating_by_cat_plot, x='category', y='product_count')
fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickfont=dict(size=14, color='black')  
    )
)

fig.show()

## map

In [89]:
map_provinceId = {
    3.0: 'EastAzarbaijan', 4.0: 'WestAzarbaijan', 5.0: 'Ardebil', 6.0: 'Esfahan', 7.0: 'Alborz',
    8.0: 'Ilam',9.0: 'Bushehr',10.0: 'Tehran',11.0: 'ChaharMahallandBakhtiari',
    12.0: 'SouthKhorasan', 13.0: 'RazaviKhorasan', 14.0: 'NorthKhorasan', 15.0:'Khuzestan',
    16.0: 'Zanjan', 17.0: 'Semnan', 18.0: 'SistanandBaluchestan', 19.0: 'Fars', 20.0: 'Qazvin',
    21.0: 'Qom', 22.0: 'Kordestan', 23.0:'Kerman', 24.0:'Kermanshah',
    25.0: 'KohgiluyehandBuyerAhmad', 26.0: 'Golestan',
    27.0: 'Gilan', 28.0: 'Lorestan', 29.0: 'Mazandaran',
    30.0: 'Markazi', 31.0: 'Hormozgan', 32.0:'Hamadan', 33.0: 'Yazd'
}

In [90]:
import geopandas as gpd

# Load the GeoJSON file
gdf = gpd.read_file('E:/Courses&Code/BaSalam-Project/data/geojson/gadm41_IRN_1.json')

In [91]:
products['vendor_provinceId'] = products['vendor_provinceId'].map(map_provinceId)

In [132]:
vendor_counts = products.groupby('vendor_provinceId')['vendor_id'].nunique()
vendor_counts = vendor_counts.reset_index(name='NAME_1')
vendor_counts.columns = ['NAME_1', 'vendor_counts']
vendor_counts['percentage'] = pd.Series(np.round(vendor_counts['vendor_counts'] / len(products['vendor_id'].unique()) * 100 , 2))
vendor_per_province = pd.merge(gdf, vendor_counts, on='NAME_1')

In [ ]:
fig = px.choropleth(vendor_per_province,
                    geojson=vendor_per_province.geometry,
                    locations=vendor_per_province.index,
                    color=vendor_per_province.vendor_counts,  
                    hover_name=vendor_per_province['NAME_1'],  
                    title="Number of vendors in each province",
                    hover_data=['percentage'])

fig.update_geos(fitbounds="locations", visible=True)

fig.show()

### worst vendor province

In [ ]:
products[(products['rating_count']>10)].groupby('vendor_id')['rating_average'].mean().sort_values(ascending=True).head(200)

In [ ]:
worst_vendor_province = products[products['vendor_id'].isin(products[(products['rating_count']>10)].groupby('vendor_id')['rating_average'].mean().sort_values(ascending=True).head(100).index)][
    ['vendor_id', 'vendor_provinceId']].drop_duplicates().groupby('vendor_provinceId')['vendor_id'].count().sort_values(ascending=False).reset_index(name='vendor_count')
worst_vendor_province.columns = ['NAME_1', 'vendor_counts']
worst_vendor_province['percentage'] = pd.Series(worst_vendor_province['vendor_counts'] )
vendor_per_province = pd.merge(gdf, worst_vendor_province, on='NAME_1')
fig = px.choropleth(vendor_per_province,
                    geojson=vendor_per_province.geometry,
                    locations=vendor_per_province.index,
                    color=vendor_per_province.vendor_counts,  
                    hover_name=vendor_per_province['NAME_1'],  
                    title="Number of vendors in each province",
                    hover_data=['percentage'])
fig.update_geos(fitbounds="locations", visible=True)

fig.show()

### best vendor province


In [ ]:
best_vendor_province = products[products['vendor_id'].isin(products[(products['rating_count']>10)].groupby('vendor_id')['rating_average'].mean().sort_values(ascending=False).head(100).index)][
    ['vendor_id', 'vendor_provinceId']].drop_duplicates().groupby('vendor_provinceId')['vendor_id'].count().sort_values(ascending=False).reset_index(name='vendor_count')
best_vendor_province.columns = ['NAME_1', 'vendor_counts']
best_vendor_province['percentage'] = pd.Series(best_vendor_province['vendor_counts'] )
vendor_per_province = pd.merge(gdf, best_vendor_province, on='NAME_1')

fig = px.choropleth(vendor_per_province,
                    geojson=vendor_per_province.geometry,
                    locations=vendor_per_province.index,
                    color=vendor_per_province.vendor_counts,  
                    hover_name=vendor_per_province['NAME_1'],  
                    title="Number of vendors in each province",
                    hover_data=['percentage'])
fig.update_geos(fitbounds="locations", visible=True)

fig.show()